**Librerias que se van a utilizar**


In [ ]:
import pandas as pd
import json
import numpy as np
import pickle
import pyarrow.parquet as pq
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Creo una funcion para normalizar la columna Category**

In [ ]:
def normalize_categories(category_list):
    if category_list is not None:
        return [category.strip() for category in category_list]
    else:
        return None

In [ ]:
def normalize_hours(hours_list):
    if hours_list is not None:
        normalized_hours = []
        for day_hours in hours_list:
            if len(day_hours) == 2:
                day, hours = day_hours
                normalized_day_hours = f"{day}: {hours}"
                normalized_hours.append(normalized_day_hours)
        return normalized_hours
    else:
        return None

In [ ]:
def flatten_dict(d):
    if d is None:
        return {}

    flattened_dict = {}
    for key, values in d.items():
        if isinstance(values, list):
            for value in values:
                flattened_dict[f"{key}: {value}"] = 1
        else:
            flattened_dict[key] = values
    return flattened_dict


In [ ]:
def cargar_y_normalizar(ruta_archivo):
    with open(ruta_archivo, 'r') as f:
        j = [json.loads(line) for line in f]
    df = pd.DataFrame(j)
    df['category'] = df['category'].apply(normalize_categories)
    df_meta = df.dropna(subset=['category'])
    df_meta = df_meta.explode('category')
    df_meta = df_meta.drop(columns=['relative_results'])
    df_meta = df_meta.drop(columns=['MISC'])
    df_meta['hours'] = df_meta['hours'].apply(normalize_hours)
    df_meta['hours'] = df_meta['hours'].apply(lambda x: ', '.join(x) if x is not None else None)
    return df_meta

**Abro los archivos de meta datacompletos**
**y normalizo la columna correspondiente**

In [ ]:
ruta_archivo1 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/1.json'
ruta_archivo2 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/2.json'
ruta_archivo3 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/3.json'
ruta_archivo4 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/4.json'
ruta_archivo5 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/5.json'
ruta_archivo6 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/6.json'
ruta_archivo7 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/7.json'
ruta_archivo8 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/8.json'
ruta_archivo9 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/9.json'
ruta_archivo10 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/10.json'
ruta_archivo11 = '/content/drive/MyDrive/Colab Notebooks/Google Maps/metadata-sitios/11.json'


df_meta1 = cargar_y_normalizar(ruta_archivo1)
df_meta2 = cargar_y_normalizar(ruta_archivo2)
df_meta3 = cargar_y_normalizar(ruta_archivo3)
df_meta4 = cargar_y_normalizar(ruta_archivo4)
df_meta5 = cargar_y_normalizar(ruta_archivo5)
df_meta6 = cargar_y_normalizar(ruta_archivo6)
df_meta7 = cargar_y_normalizar(ruta_archivo7)
df_meta8 = cargar_y_normalizar(ruta_archivo8)
df_meta9 = cargar_y_normalizar(ruta_archivo9)
df_meta10 = cargar_y_normalizar(ruta_archivo10)
df_meta11 = cargar_y_normalizar(ruta_archivo11)

**indico que categoria quiero**
 importante colocarlo en ingles
y concatenamos para tener solo un archivo df

In [ ]:
df_meta1['category']

0                      Pharmacy
1              Textile exporter
2             Korean restaurant
3                  Fabric store
4                  Fabric store
                  ...          
274999        Kitchen remodeler
274999             Lumber store
274999              Paint store
274999    Plumbing supply store
275000         Auto repair shop
Name: category, Length: 530472, dtype: object

'Tourist attraction'

In [ ]:
categoria_especifica = 'Tourist attraction'

In [ ]:
df_meta_n1 = df_meta1[df_meta1['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n2 = df_meta2[df_meta2['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n3 = df_meta3[df_meta3['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n4 = df_meta4[df_meta4['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n5 = df_meta5[df_meta5['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n6 = df_meta6[df_meta6['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n7 = df_meta7[df_meta7['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n8 = df_meta8[df_meta8['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n9 = df_meta9[df_meta9['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n10 = df_meta10[df_meta10['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n11 = df_meta11[df_meta11['category'].str.contains(categoria_especifica, case=False, na=False)]

#se concatena toda la meta data
df_total_meta = pd.concat([df_meta_n1, df_meta_n2, df_meta_n3, df_meta_n4, df_meta_n5, df_meta_n6, df_meta_n7, df_meta_n8, df_meta_n9, df_meta_n10, df_meta_n11], ignore_index=True)

#se guarda el MetaData en CSV con la categoria seleccionada
df_total_meta.to_csv('/content/drive/My Drive/Colab Notebooks/metadata.csv', index=False)

#guardamos otro CSV mas pequeños para el Machine Learning
columnas_seleccionadas = ['gmap_id', 'price', 'num_of_reviews', 'avg_rating']
df_seleccionado_ml = df_total_meta[columnas_seleccionadas]
df_seleccionado_ml.to_csv('/content/drive/My Drive/Colab Notebooks/MetaData ML.csv', index=False)

**Esta variable solo guarda la columns de gmap_id para luego asi solo guarda los reviews que necesitamos en estado**

In [ ]:
df_gmap_id = df_total_meta['gmap_id']


**Abro los archivos de review de cualquier estado y lo guardo en un solo DF**

In [ ]:
estado = 'Florida'

In [ ]:
directorio = f'/content/drive/MyDrive/Colab Notebooks/Google Maps/reviews-estados/review-{estado}'
df_estado = pd.DataFrame()

def limpiar_datos(line):
    try:
        # Reemplazar caracteres de escape por espacios en blanco
        line = line.replace('\\', ' ')
        return json.loads(line)
    except json.JSONDecodeError:
        return None

for archivo_json in os.listdir(directorio):
    if archivo_json.endswith('.json'):
        ruta_completa = os.path.join(directorio, archivo_json)
        with open(ruta_completa, 'r', encoding='utf-8') as f:
            datos = [limpiar_datos(line) for line in f if line.strip()]
            datos = [d for d in datos if d is not None]  # Eliminar datos nulos
            df_temp = pd.DataFrame(datos)

            if 'pics' in df_temp.columns:
                df_temp.drop('pics', axis=1, inplace=True)
            if 'resp' in df_temp.columns:
                df_temp.drop('resp', axis=1, inplace=True)
            # Rellenar valores nulos con ceros
            df_temp.fillna(0, inplace=True)

            df_estado = pd.concat([df_estado, df_temp], ignore_index=True)


**Guardo el estado en un CSV temporal**

In [ ]:
#del estado solo busco y concatelo los gmap_id que estan en MetaData
df_resultado = df_estado.merge(df_gmap_id, on='gmap_id', how='inner')

#se guarda el archico en CSV de estado
df_resultado.to_csv('/content/drive/My Drive/Colab Notebooks/estado.csv', index=False)

# creamos un CSV mas pequeño para el Machine Learning de estado
columnas_seleccionadas_estado = ['user_id', 'rating', 'text', 'gmap_id']
df_seleccionado_estado = df_resultado[columnas_seleccionadas_estado]
df_seleccionado_estado.to_csv('/content/drive/My Drive/Colab Notebooks/Estado-ML.csv', index=False)



**Abro los archivos de Yelp**

In [ ]:
archivo_pkl = '/content/drive/MyDrive/Colab Notebooks/Yelp/business.pkl'
with open(archivo_pkl, 'rb') as archivo:
    business = pickle.load(archivo)
business = business.loc[:, ~business.columns.duplicated()]
business = business.drop(columns=['attributes'])
business = business.drop(columns=['hours'])

In [ ]:
business['categories']

0         Doctors, Traditional Chinese Medicine, Naturop...
1         Shipping Centers, Local Services, Notaries, Ma...
2         Department Stores, Shopping, Fashion, Home & G...
3         Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                                 Brewpubs, Breweries, Food
                                ...                        
150341                           Nail Salons, Beauty & Spas
150342    Pets, Nurseries & Gardening, Pet Stores, Hobby...
150343    Shopping, Jewelry, Piercing, Toy Stores, Beaut...
150344    Fitness/Exercise Equipment, Eyewear & Optician...
150345    Beauty & Spas, Permanent Makeup, Piercing, Tattoo
Name: categories, Length: 150346, dtype: object

**Selecciono la categoria de yelp que deseo seleccionar al igual de meta data**

In [ ]:
categoria_especifica_yelp = 'recreation'

In [ ]:
business = business[business['categories'].str.contains(categoria_especifica_yelp, case=False, na=False)]


In [ ]:
business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
528,kxvOUehCtJAL2nYKkeiR-w,Incrediplex,6002 Sunnyside Rd,Indianapolis,NJ,46236,39.866083,-85.975098,4.0,33,0,"Specialty Schools, Education, Gyms, Sports Clu..."
615,82suwumWp1MEq04QhiS9DQ,Surreal Virtual Reality Studio,"436 State St, Ste B",Santa Barbara,MO,93101,34.416782,-119.695139,5.0,30,0,"Recreation Centers, Arts & Entertainment, Virt..."
635,xHspDrdyq1g27yRDezB5yA,Kaiserman JCC,45 Haverford Rd,Philadelphia,PA,19096,39.982542,-75.268542,3.5,11,1,"Preschools, Recreation Centers, Education, Gym..."
813,6jomGWEI4ryImrWPguUQQQ,Santa Barbara Athletic Club,520 Castillo St,Santa Barbara,NJ,93101,34.413384,-119.700341,3.0,55,1,"Fitness & Instruction, Swimming Pools, Active ..."
1915,vfBzDd-ceWnCrKqHdZwMSg,Wasting Arrows,5260 Longley Ln,Reno,PA,89502,39.474317,-119.7637,4.5,93,1,"Sporting Goods, Archery, Active Life, Recreati..."
...,...,...,...,...,...,...,...,...,...,...,...,...
144674,v-GUuEdEQ4RLcdbjUy84Kg,CSA Palm Harbor,1500 16th St,Palm Harbor,IL,34683,28.08153,-82.757815,3.5,6,1,"Recreation Centers, Active Life"
145224,WtmmtVkui7AjVIZvnqQHwg,Get Air King of Prussia,555 S Henderson Rd,King of Prussia,FL,19406,40.080087,-75.356335,2.5,85,0,"Trampoline Parks, Active Life, Recreation Centers"
145817,Q0mebaho7udFFvKNn36t1w,Post Road Recreation Center,4700 N Post Rd,Indianapolis,FL,46226,39.841741,-86.008825,2.5,9,0,"Recreation Centers, Haunted Houses, Active Lif..."
147482,Arx7AUZaiY6v-E9jNw8bYw,Richard A. Carlucci Recreation And Aquatic Center,651 Vestal Rd,Plainfield,PA,46168,39.708806,-86.412776,4.5,10,1,"Active Life, Recreation Centers, Swimming Pools"


In [ ]:
with open ('/content/drive/MyDrive/Colab Notebooks/Yelp/checkin.json','r') as f:
  jcheckin = [json.loads(line) for line in f]

df_checkin_Yelp= pd.DataFrame(jcheckin)

In [ ]:
df_checkin_Yelp

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"2013-03-23 16:22:47, 2013-04-07 02:03:12, 2013..."
131926,zznZqH9CiAznbkV6fXyHWA,2021-06-12 01:16:12
131927,zzu6_r3DxBJuXcjnOYVdTw,"2011-05-24 01:35:13, 2012-01-01 23:44:33, 2012..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"2016-12-03 23:33:26, 2018-12-02 19:08:45"


**Combino la tabla de Business y chekin mediante Business_id y se guarda solo los datos que corresponde a la categoria seleccionada**


In [ ]:
combined_df = df_checkin_Yelp.merge(business, on='business_id', how='inner')

#guardo el documento en CSV
combined_df.to_csv('/content/drive/My Drive/Colab Notebooks/yelp checkin business.csv', index=False)

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_seleccionadas_bc = ['business_id', 'date', 'state', 'is_open']
df_seleccionado_bc = combined_df[columnas_seleccionadas_bc]
df_seleccionado_bc.to_csv('/content/drive/My Drive/Colab Notebooks/Yelp Checkin Business-ML.csv', index=False)

**Abro el archico review y se concatena con la tabla business mediante business_id y se guarda los review de la categoria seleccionada**

In [ ]:
review = '/content/drive/MyDrive/Colab Notebooks/Yelp/review.json'
df_review = pd.DataFrame()

def limpiar_datos(line):
    try:
        line = line.replace('\\', ' ')
        return json.loads(line)
    except json.JSONDecodeError:
        return None

with open(review, 'r', encoding='utf-8') as f:
    datos = [limpiar_datos(line) for line in f if line.strip()]
    datos = [d for d in datos if d is not None]  # Eliminar datos nulos
    df_temp = pd.DataFrame(datos)

    if 'stars' in df_temp.columns:
        df_temp.drop('stars', axis=1, inplace=True)
    if 'cool' in df_temp.columns:
        df_temp.drop('cool', axis=1, inplace=True)
    if 'funny' in df_temp.columns:
        df_temp.drop('funny', axis=1, inplace=True)
    if 'date' in df_temp.columns:
        df_temp.drop('date', axis=1, inplace=True)

    # Rellenar valores nulos con ceros
    df_temp.fillna(0, inplace=True)

    df_review = pd.concat([df_review, df_temp], ignore_index=True)


In [ ]:
#combinamos desde la tabla businnes los reviwes q tenga el mismo business_id de ellos
combined_df_business_reviwes = df_review.merge(business, on='business_id', how='inner')
combined_df_business_reviwes.to_csv('/content/drive/My Drive/Colab Notebooks/yelp review business.csv', index=False, encoding='utf-8')

cambio_Ml = combined_df_business_reviwes

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_a_mantener = ['business_id', 'state', 'stars', 'text']
cambio_Ml = cambio_Ml[columnas_a_mantener]
cambio_Ml.to_csv('/content/drive/My Drive/Colab Notebooks/Yelp Reviwes Business-ML.csv', index=False)


**Abro el archico tip y se concatena con la tabla business mediante business_id y se guarda los tip de la categoria seleccionada**


In [ ]:
tip = '/content/drive/MyDrive/Colab Notebooks/Yelp/tip.json'
df_tip = pd.DataFrame()

def limpiar_datos(line):
    try:
        line = line.replace('\\', ' ')
        return json.loads(line)
    except json.JSONDecodeError:
        return None

with open(tip, 'r', encoding='utf-8') as f:
    datos = [limpiar_datos(line) for line in f if line.strip()]
    datos = [d for d in datos if d is not None]  # Eliminar datos nulos
    df_temp = pd.DataFrame(datos)

    if 'compliment_count' in df_temp.columns:
        df_temp.drop('compliment_count', axis=1, inplace=True)
    if 'date' in df_temp.columns:
        df_temp.drop('date', axis=1, inplace=True)

    # Rellenar valores nulos con ceros
    df_temp.fillna(0, inplace=True)

    df_tip = pd.concat([df_tip, df_temp], ignore_index=True)


In [ ]:
#combino la tabla tip con business para saber que datos necesito de tip
combined_df_business_tip = df_tip.merge(business, on='business_id', how='inner')

#lo guardo en un CSV
combined_df_business_tip.to_csv('/content/drive/My Drive/Colab Notebooks/yelp tip business.csv', index=False)

cambio_tip_Ml = combined_df_business_tip

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_a_mantener = ['business_id', 'text', 'state', 'categories']
cambio_tip_Ml = cambio_tip_Ml[columnas_a_mantener]
cambio_tip_Ml.to_csv('/content/drive/My Drive/Colab Notebooks/Yelp Tip Business-ML.csv', index=False)

-------------------------------------------
este archico el ultimo esta arrojando problemas pero bueno lo dejamos ahi por las dudas


**Abro el archico user y se concatena con la tabla tip mediante user_id y se guarda los user de la categoria seleccionada**


In [ ]:
archivo_parquet = '/content/drive/MyDrive/Colab Notebooks/Yelp/user.parquet'
tabla_parquet = pq.read_table(archivo_parquet)
user = tabla_parquet.to_pandas()

In [ ]:
user

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105592,4QGxxakRZeOlg_qDuxmTeQ,Jennilee,38,2012-01-19 23:33:02,74,9,6,,kmwNG5LZSHFmveg6wYYdrw,0,...,1,0,0,0,1,4,0,0,1,0
2105593,tmelBbVBGAzXBVfH2u_R6g,Gerry,19,2009-06-09 16:34:54,14,5,2,,"BFYdCAMFyjYHDwesndEXEg, _9fTIqfSJc7g3V_o76XRVg...",1,...,1,0,0,0,0,1,0,0,0,0
2105594,tpBznnD6uJN3m_pJubj09w,Emily,26,2013-08-13 23:18:11,4,1,2,,"bKV3ly2MuK-K1cptMrFknQ, liel18zRoSB4tEkUP7i6Cg...",0,...,0,0,0,0,1,0,0,0,0,0
2105595,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
combined_df_tip_user = user.merge(combined_df_business_tip, on='user_id', how='inner')

In [ ]:
combined_df_tip_user.to_csv('/content/drive/My Drive/Colab Notebooks/yelp user tip.csv', index=False)

In [ ]:
combined_df_tip_user

,user_id,name_x,review_count_x,yelping_since,useful,funny,cool,elite,friends,fans,...,address,city,state,postal_code,latitude,longitude,stars,review_count_y,is_open,categories
0,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,1401 Clark Ave,Saint Louis,NJ,63103,38.626917,-90.202488,4.0,16,1,"Professional Sports Teams, Event Planning & Se..."
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,1401 Clark Ave,Saint Louis,NJ,63103,38.626917,-90.202488,4.0,16,1,"Professional Sports Teams, Event Planning & Se..."
2,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,1302 Gallatin Ave,Nashville,FL,37206,36.193147,-86.744564,4.5,63,1,"Venues & Event Spaces, Event Planning & Servic..."
3,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,1302 Gallatin Ave,Nashville,FL,37206,36.193147,-86.744564,4.5,63,1,"Venues & Event Spaces, Event Planning & Servic..."
4,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,1401 Clark Ave,Saint Louis,NJ,63103,38.626917,-90.202488,4.0,16,1,"Professional Sports Teams, Event Planning & Se..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,CfNY3VpJECGoJKm2rSCgHA,Terrance,2,2019-05-25 15:16:04,2,0,0,,None,0,...,"1719 Washington Ave, Fl 2",Philadelphia,TN,19146,39.93883,-75.17232,5.0,153,1,"Local Flavor, Recreation Centers, Amateur Spor..."
1739,rNHWNfl70xSc6MelxOn-oQ,Audrey,6,2012-04-19 23:56:00,2,0,1,,None,0,...,555 S Henderson Rd,King of Prussia,FL,19406,40.080087,-75.356335,2.5,85,0,"Trampoline Parks, Active Life, Recreation Centers"
1740,XF4Hlwa8v7Yto_NPeilpaA,Matt,2,2012-05-13 03:52:08,6,4,4,,None,0,...,555 S Henderson Rd,King of Prussia,FL,19406,40.080087,-75.356335,2.5,85,0,"Trampoline Parks, Active Life, Recreation Centers"
1741,EsCk7btRfGlgH4cgeb4DPg,Pete,1,2018-04-19 12:18:58,0,0,0,,None,0,...,"1719 Washington Ave, Fl 2",Philadelphia,TN,19146,39.93883,-75.17232,5.0,153,1,"Local Flavor, Recreation Centers, Amateur Spor..."
